# Plot output h5 data generated via ddcurves



In [1]:
import h5py
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt

from iwaves import kdv
from iwaves.utils import isw
from tqdm import tqdm # progress bar
from glob import glob



/home/suntans/.conda/envs/soda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The mpl_toolkits.axes_grid module was deprecated in version 2.1. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [16]:
#############3
# Inputs

#f = h5py.File('DATA_QC/test_beta-samples.h5','r')
#h5file = 'DATA_QC/IMOS_Density_KIM200_2013_a_beta-samples-array-all-data.h5'

datadir = '/home/suntans/cloudstor/Data/IMOS/'
datafile1 = '%s/IMOS_Density_KIM400_2014_a_density_bhm_6params_2018-11-27.h5'%datadir
#datafile2 = '%s/DATA_QC/Crux_KP150_Phs2_Density_lowpass'%datadir
sitename = 'ITFTIS'
zmin = -405
nparams = 6


dz = 10.0
mode = 0
# top down
zout = np.arange(0,zmin, -dz)
zout

array([   0.,  -10.,  -20.,  -30.,  -40.,  -50.,  -60.,  -70.,  -80.,
        -90., -100., -110., -120., -130., -140., -150., -160., -170.,
       -180., -190., -200., -210., -220., -230., -240., -250., -260.,
       -270., -280., -290., -300., -310., -320., -330., -340., -350.,
       -360., -370., -380., -390., -400.])

In [17]:
glob('%s/*_%s_*_%dparams_*.h5'%(datadir, sitename, nparams))
#'%s/IMOS_Density_%s_*_%dparmas_*.h5'%(datadir, sitename, nparams)

['/home/suntans/cloudstor/Data/IMOS/IMOS_Density_ITFTIS_2013_b_density_bhm_6params_2018-11-29.h5',
 '/home/suntans/cloudstor/Data/IMOS/IMOS_Density_ITFTIS_2014_a_density_bhm_6params_2018-11-29.h5',
 '/home/suntans/cloudstor/Data/IMOS/IMOS_Density_ITFTIS_2014_b_density_bhm_6params_2018-11-29.h5',
 '/home/suntans/cloudstor/Data/IMOS/IMOS_Density_ITFTIS_2015_a_density_bhm_6params_2018-11-29.h5',
 '/home/suntans/cloudstor/Data/IMOS/IMOS_Density_ITFTIS_2015_b_density_bhm_6params_2018-11-28.h5']

# Load

Data is stored in hdf5 format. The $\beta$'s are stored in the group `data/beta_samples` that has dimensions [#parameters, #time, #samples]

In [4]:
f = h5py.File(datafile1,'r')
for kk in f.keys():
    print(kk)
print(f['depth'][:].min())
f.close()

beta_samples
depth
mu_beta_samples
rho
time
-406.2


In [5]:
def load_density_h5(h5file):
    f = h5py.File(h5file,'r')
    data = f['beta_samples'][:]
    time = f['time'][:].astype('<M8[ns]')
    f.close()
    return data,time


data1,time = load_density_h5('%s'%datafile1)
#data2 = load_density_h5('%s_beta-samples-array-all-data.h5'%datafile2)

#data = np.concatenate([data1,data2], axis=1)
data = data1
nparam, nt, ntrace = data[:].shape
print(nparam, nt, ntrace)


6 811 500


In [22]:
def double_tanh_6(beta, z):
    
    return beta[0] - beta[1]*(np.tanh((z+beta[2])/beta[3])
                + np.tanh((z+beta[2] + beta[4])/beta[5]))

def double_tanh_7(beta, z):
    
    return beta[0] - beta[1]*np.tanh((z+beta[2])/beta[3]) \
            - beta[6]* np.tanh((z+beta[2] + beta[4])/beta[5])

In [7]:
#%%time

samples = ntrace
alpha_ens = np.zeros((nt,samples))
c_ens = np.zeros((nt,samples))

rand_loc = np.random.randint(0, ntrace, samples)

for tstep in tqdm(range(0,nt)):
    #if (tstep%20==0):
    #    print('%d of %d...'%(tstep,nt))
    for ii in range(samples):
        rhotmp = double_tanh_6(data[:,tstep, rand_loc[ii]], zout)

        N2 = -9.81/1000*np.gradient(rhotmp,-dz)

        phi,cn = isw.iwave_modes(N2, dz)
        
        phi_1 = phi[:,mode]
        phi_1 =phi_1 / np.abs(phi_1).max()
        phi_1 *= np.sign(phi_1.sum())
        
        alpha = isw.calc_alpha(phi_1, cn[mode],N2,dz)

        alpha_ens[tstep,ii] = alpha
        c_ens[tstep,ii] = cn[mode]
        #mykdv = kdv.KdV(rhotmp,zout)


  2%|▏         | 19/811 [00:48<33:25,  2.53s/it]

KeyboardInterrupt: 

In [8]:
# Create an xray dataset with the output
dims2 = ('time','ensemble',)
dims3 = ('params','time','ensemble')

#time = rho.time.values
#time = range(nt)
coords2 = {'time':time, 'ensemble':range(ntrace)}
coords3 = {'time':time, 'ensemble':range(ntrace), 'params':range(nparams)}
           

cn_da = xr.DataArray(c_ens,
    coords=coords2,
    dims=dims2,
    attrs={'long_name':'', 'units':''},
    )

alpha_da = xr.DataArray(alpha_ens,
    coords=coords2,
    dims=dims2,
    attrs={'long_name':'', 'units':''},
    )

beta_da = xr.DataArray(data,
    coords=coords3,
    dims=dims3,
    attrs={'long_name':'', 'units':''},
    )

dsout = xr.Dataset({'cn':cn_da, 'alpha':alpha_da, 'beta':beta_da,\
                   })
dsout

<xarray.Dataset>
Dimensions:   (ensemble: 500, params: 6, time: 811)
Coordinates:
  * time      (time) datetime64[ns] 2014-02-01T12:00:01 ... 2014-08-23T00:00:01
  * ensemble  (ensemble) int64 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
  * params    (params) int64 0 1 2 3 4 5
Data variables:
    cn        (time, ensemble) float64 1.498 1.508 1.519 1.507 ... 0.0 0.0 0.0
    alpha     (time, ensemble) float64 -0.00914 -0.009538 -0.009237 ... 0.0 0.0
    beta      (params, time, ensemble) float64 1.024e+03 1.023e+03 ... 113.3

In [19]:
def calc_nliw_params(h5file, zmin, dz):
    
    # Lload the data
    data,time = load_density_h5(h5file)
    nparam, nt, ntrace = data[:].shape
    
    zout = np.arange(0,zmin, -dz)

    
    # Calculate c and alpha
    samples = ntrace
    alpha_ens = np.zeros((nt,samples))
    c_ens = np.zeros((nt,samples))

    rand_loc = np.random.randint(0, ntrace, samples)

    for tstep in tqdm(range(0,nt)):
        #if (tstep%20==0):
        #    print('%d of %d...'%(tstep,nt))
        for ii in range(samples):
            
            if nparams == 6:
                rhotmp = double_tanh_6(data[:,tstep, rand_loc[ii]], zout)
            elif nparams == 7:
                rhotmp = double_tanh_7(data[:,tstep, rand_loc[ii]], zout)


            N2 = -9.81/1000*np.gradient(rhotmp,-dz)

            phi,cn = isw.iwave_modes(N2, dz)

            phi_1 = phi[:,mode]
            phi_1 =phi_1 / np.abs(phi_1).max()
            phi_1 *= np.sign(phi_1.sum())

            alpha = isw.calc_alpha(phi_1, cn[mode],N2,dz)

            alpha_ens[tstep,ii] = alpha
            c_ens[tstep,ii] = cn[mode]
            #mykdv = kdv.KdV(rhotmp,zout)
            
    # Export to an xarray data set
    # Create an xray dataset with the output
    dims2 = ('time','ensemble',)
    dims3 = ('params','time','ensemble')

    #time = rho.time.values
    #time = range(nt)
    coords2 = {'time':time, 'ensemble':range(ntrace)}
    coords3 = {'time':time, 'ensemble':range(ntrace), 'params':range(nparams)}


    cn_da = xr.DataArray(c_ens,
        coords=coords2,
        dims=dims2,
        attrs={'long_name':'', 'units':''},
        )

    alpha_da = xr.DataArray(alpha_ens,
        coords=coords2,
        dims=dims2,
        attrs={'long_name':'', 'units':''},
        )

    beta_da = xr.DataArray(data,
        coords=coords3,
        dims=dims3,
        attrs={'long_name':'', 'units':''},
        )

    dsout = xr.Dataset({'cn':cn_da, 'alpha':alpha_da, 'beta':beta_da,})
    
    return dsout


In [25]:
calc_nliw_params(datafile1, -400, 10)



  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 1/811 [00:02<34:28,  2.55s/it]

  0%|          | 2/811 [00:04<32:57,  2.44s/it]

  0%|          | 3/811 [00:07<33:48,  2.51s/it]

  0%|          | 4/811 [00:09<33:10,  2.47s/it]

  1%|          | 5/811 [00:12<33:11,  2.47s/it]

  1%|          | 6/811 [00:14<33:25,  2.49s/it]

  1%|          | 7/811 [00:17<32:43,  2.44s/it]

  1%|          | 8/811 [00:19<32:50,  2.45s/it]

  1%|          | 9/811 [00:22<32:50,  2.46s/it]

  1%|          | 10/811 [00:24<32:48,  2.46s/it]

  1%|▏         | 11/811 [00:27<32:52,  2.47s/it]

  1%|▏         | 12/811 [00:29<32:46,  2.46s/it]

  2%|▏         | 13/811 [00:32<33:12,  2.50s/it]

  2%|▏         | 14/811 [00:35<33:37,  2.53s/it]

  2%|▏         | 15/811 [00:38<34:01,  2.56s/it]

  2%|▏         | 16/811 [00:41<34:02,  2.57s/it]

  2%|▏         | 17/811 [00:43<34:02,  2.57s/it]

  2%|▏         | 18/811 [00:46<33:56,  2.57s/it]

  2%|▏         | 19/811 [00:48<33:46,  2.56s/it]

KeyboardInterrupt: 